In [8]:
import numpy as np
import math

In [13]:
def fit(X, y, lmbda):
    N, D = X.shape
    cs = np.unique(y)
    C = len(cs)
    mu = np.mean(X, axis = 0)
    s2 = np.zeros(D)

    for c in cs:
        i = y == c
        X_c = X[i]
        prior = np.mean(i)
        mu_c = np.mean(X_c, axis = 0)
        for x in X_c:
            s2 += (x - mu_c) ** 2 / (1.0 * N - C)
    
    for c in cs:
        i = y == c
        X_c = X[i]
        prior = np.mean(i)
        mu_c = np.mean(X_c, axis = 0)
        
    return prior, mu, sigma2
        
def predict(model, X):
    prior, mu, sigma2 = model
    delta_c = math.log(prior) - np.sum([X[:, j] - mu[c, j] / (2 * sigma2[j]) for j in range(D)])
    X = np.hstack((X, np.ones((len(X), 1))))
    cs, Beta = model
    return np.array([cs[i] for i in np.argmax(X.dot(Beta), axis = 1)])

In [14]:
X = np.array([
    [2, 3, 4, 5],
    [2.2, 3.1, 4.2, 5.3],
    [2.1, 3.2, 4.1, 5.2],
])

y = np.array([1, 1, 2])

In [15]:
model = fit(X, y, 0.1)
#np.mean(predict(model, X) == y)

1049.85694327
